# Carregando e visualizando dados no Google Colab

Vamos começar entendendo como um LLM (grande modelo de linguagem) pode nos auxiliar, por exemplo, gerando um código para analisar um arquivo. Iniciaremos no Google Colab carregando os dados da empresa. Para isso, importaremos a biblioteca Pandas, voltada para análise de dados, com o comando:

In [21]:
import pandas as pd
from dotenv import load_dotenv
import os


load_dotenv()
 
GOOGLE_API_KEY = str(os.getenv("GOOGLE_API_KEY"))

In [2]:
print("Google API Key:", len(GOOGLE_API_KEY))

Google API Key: 39


In [ ]:
url = 'https://raw.githubusercontent.com/Ricardo-Filgueiras/AgentesIA-com-Analise-de-dados/refs/heads/main/database/dados_entregas.csv' 

In [ ]:
df = pd.read_csv(url)

In [23]:
df.head(n=3)

,ID_pedido,anos_experiencia_agente,classificacao_agente,latitude_loja,longitude_loja,latitude_entrega,longitude_entrega,data_pedido,hora_pedido,hora_retirada,clima,trafego,veiculo,area,categoria_produto,tempo_entrega
0,ialx566343618,8,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Ensolarado,Alto,Motocicleta,Urbano,Roupas,120
1,akqg208421122,10,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Chuvoso,Congestionamento,Scooter,Metropolitano,Eletronicos,165
2,njpu434582536,16,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Tempestade,Baixo,Motocicleta,Urbano,Esportes,130


# Automatizando a análise de dados com LLMs

O objetivo é que a equipe de dados, ao receber esse arquivo, possa construir um modelo preditivo para prever o tempo de entrega com base nas características disponíveis. Para isso, é necessário analisar os dados, verificar os tipos de cada coluna, tratar dados nulos e duplicados, e explorar relações, como a influência dos anos de experiência no tempo de entrega.

Queremos automatizar esse processo, evitando a necessidade de escrever código manualmente para calcular correlações, por exemplo. Desejamos que um LLM gere o código necessário para automatizar essas tarefas. Vamos aprender a obter esse código utilizando a ferramenta LangChain, que está em alta no campo da inteligência artificial e é útil para criar soluções e aplicações.

Configurando o LangChain e o Gemini

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(
     model="gemini-2.0-flash",
     temperature=0,
     max_tokens=None,
     timeout=None,
     max_retries=2,
     
    )

In [6]:
messages = [
    (
        "system",
        "Você é um Analista de dados prestativo que procusar padrões e ingsiths usando as boas praticas.",
    ),
    ("human",     """ Eu tenho um dataframe chamado 'df' com as colunas 'anos_experiencia_agente' e 'tempo_entrega'.
        Escreva o código Python com a biblioteca Pandas para calcular a correlação entre as duas colunas.
        Retorne o Markdown para o trecho de código Python e nada mais.
        """
        ),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="```python\n import pandas as pd\n \n\n # Supondo que seu DataFrame já esteja carregado como 'df'\n \n\n # Calcula a correlação entre as colunas 'anos_experiencia_agente' e 'tempo_entrega'\n correlacao = df['anos_experiencia_agente'].corr(df['tempo_entrega'])\n \n\n print(correlacao)\n ```", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d2328cae-8bec-4367-b5ba-7dde57a0a48c-0', usage_metadata={'input_tokens': 91, 'output_tokens': 86, 'total_tokens': 177, 'input_token_details': {'cache_read': 0}})

In [7]:
print(ai_msg.content)

```python
 import pandas as pd
 

 # Supondo que seu DataFrame já esteja carregado como 'df'
 

 # Calcula a correlação entre as colunas 'anos_experiencia_agente' e 'tempo_entrega'
 correlacao = df['anos_experiencia_agente'].corr(df['tempo_entrega'])
 

 print(correlacao)
 ```


## Executando o código gerado com ferramentas Python

primeiramente vamos instalar mais uma ferramenta !

Para automatizar o processo de execução de código gerado por um LLM, utilizamos o LangChain, especificamente a ferramenta python-est-repltool. Vamos começar instalando o pacote experimental do LangChain, onde essa ferramenta está localizada. Para isso, executamos o seguinte comando

In [9]:
# !pip install langchain-experimental -q

In [10]:
from langchain_experimental.tools import PythonAstREPLTool

Configurando a ferramenta PythonAstREPLTool

In [11]:
ferramenta_python = PythonAstREPLTool(locals={"df": df})

In [12]:
ferramenta_python.invoke("df['anos_experiencia_agente'].corr(df['tempo_entrega'])")

np.float64(-0.2516410755060903)

### Integrando a ferramenta com um LLM

In [14]:
llm_com_ferramenta = llm.bind_tools([ferramenta_python], tool_choice=ferramenta_python.name)

In [19]:
resposta = llm_com_ferramenta.invoke(
    """Eu tenho um dataframe 'df' e quero saber a correlação entre as colunas 'anos_experiencia_agente' 
    e 'tempo_entrega'"""
)

In [15]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=ferramenta_python.name, first_tool_only=True)

In [16]:
cadeia = llm_com_ferramenta | parser

In [18]:
cadeia.invoke(
    """Eu tenho um dataframe 'df' e quero saber a correlação entre as colunas 'anos_experiencia_agente' 
    e 'tempo_entrega'
    
    """
)

{'query': 'import pandas as pd\n\ndata = {"anos_experiencia_agente": [1, 2, 3, 4, 5], "tempo_entrega": [5, 4, 3, 2, 1]}\ndf = pd.DataFrame(data)\ncorrelation = df["anos_experiencia_agente"].corr(df["tempo_entrega"])\nprint(correlation)\n'}

## Criando o prompt com template

Para orientar o modelo, vamos criar um prompt usando um template. No LangChain, temos algumas opções, como o ChatPromptTemplate, que serve para criar um prompt de conversação. Imagine que haverá uma conversa em que a pessoa envia uma mensagem, a inteligência artificial gera o código, e esse código é executado pela nossa ferramenta Python. Todo esse processo ocorre de forma integrada.

In [25]:
from langchain_core.prompts import ChatPromptTemplate

system = f"""Você tem acesso a um dataframe pandas `df`. \
Aqui está a saída de `df.head().to_markdown()`:

\`\`\`
    {df.head().to_markdown()}
\`\`\`

Dada uma pergunta do usuário, escreva o código Python para respondê-la. \
Retorne SOMENTE o código Python válido e nada mais. \
Não presuma que você tem acesso a nenhuma biblioteca além das bibliotecas Python integradas e pandas."""


prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

cadeia = prompt | llm_com_ferramenta | parser | ferramenta_python

resposta = cadeia.invoke({"question": "Qual é a correlação entre anos de experiência do agente e tempo de entrega?"})
print(resposta)

<>:12: SyntaxWarning: invalid escape sequence '\`'
<>:12: SyntaxWarning: invalid escape sequence '\`'
<>:12: SyntaxWarning: invalid escape sequence '\`'
<>:12: SyntaxWarning: invalid escape sequence '\`'
C:\Users\Campeao Lub\AppData\Local\Temp\ipykernel_14188\1920347851.py:12: SyntaxWarning: invalid escape sequence '\`'
  Não presuma que você tem acesso a nenhuma biblioteca além das bibliotecas Python integradas e pandas."""
C:\Users\Campeao Lub\AppData\Local\Temp\ipykernel_14188\1920347851.py:12: SyntaxWarning: invalid escape sequence '\`'
  Não presuma que você tem acesso a nenhuma biblioteca além das bibliotecas Python integradas e pandas."""


-0.2516410755060903



In [26]:
resposta = cadeia.invoke({"question": "Qual é média do tempo de entrega para cada tipo de clima?"})
print(resposta)

clima
Chuvoso       123.211825
Ensolarado    103.664453
Neblina       136.570833
Nublado       138.286773
Tempestade    123.238509
Ventania      123.658037
Name: tempo_entrega, dtype: float64

